# measure weights for mulitple epochs

In [ ]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import train
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-s', '--seed',
    type=float,
    default=1,
    help="Seed for randomness",
    dest="seed"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args = parser.parse_args()
model_str = args.name
layer_str = args.layer
tf.random.set_seed(args.seed)

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
# for 5000 epochs use "measure_epochs_only=True"
model, weights = train.weight_measure(
    model, layer_str, 10, final_learning_rate, x_train, y_train, batch_size)
np.save(model_str+'/weights_'+layer_str, weights)


# compute the principal components

In [ ]:

import numpy as np
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
args = parser.parse_args()
model_str = args.name
layer_str = args.layer

weights = np.load(model_str+'/weights_'+layer_str+'.npy')
Cov = np.cov(weights.T)
variance, pcomp_cov = np.linalg.eigh(Cov)
pcomp = pcomp_cov.T[::-1]
np.save(model_str+'/pc_'+layer_str, pcomp)


# compute theta

In [ ]:

import numpy as np
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
args = parser.parse_args()
model_str = args.name
layer_str = args.layer

weights = np.load(model_str+'/weights_'+layer_str+'.npy')
pcomp = np.load(model_str+'/pc_'+layer_str+'.npy')
theta = np.tensordot(weights, pcomp, axes=[1, 1])
np.save(model_str+'/theta_'+layer_str, theta.T)


# compute singular value field

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import analysis
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, all or layer of sv",
    dest="layer"
)
parser.add_argument(
    '-m', '--mode',
    type=str,
    default="evh",
    help="type of input vectors, evh or pc",
    dest="mode"
)

parser.add_argument(
    '-lx', '--lyrdex',
    type=int,
    default="-1",
    help="layer index for singular values",
    dest="lx"
)

args = parser.parse_args()
model_str = args.name
md_str = args.mode
layer_index = args.lx
layer_str = args.layer


exec(open(model_str+'/'+model_str+".py").read())
field = analysis.sv_field(model, model_str, layer_str, md_str,layer_index)
if md_str == "evh":
    if layer_str == "all":
        np.save(model_str+'/field_'+str(layer_index), field)
    else:
        np.save(model_str+'/field_'+layer_str, field)
else:
    np.save(model_str+'/field_'+layer_str+'_'+md_str, field)


# compute accuracies and losses for different number of components added

In [ ]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import analysis
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
parser.add_argument(
    '-m', '--mode',
    type=str,
    default="evh",
    help="type of input vectors, evh or pc",
    dest="mode"
)
parser.add_argument(
    '-a', '--abs',
    type=bool,
    default=True,
    help="whether eigenvalues should be added in order of magnitude or sign",
    dest="abs"
)

args = parser.parse_args()
model_str = args.name
md_str = args.mode
layer_str = args.layer


exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
if md_str == "evh":
    if args.abs:
        sort_list = np.argsort(
            np.abs(np.load(model_str+'/ewh_'+layer_str+'.npy')))
        vec = np.take(np.load(model_str+'/evh_'+layer_str+'.npy'),
                      sort_list, axis=0)[::-1]
    else:
        vec = np.load(model_str+'/evh_'+layer_str+'.npy')
else:
    vec = np.load(model_str+'/'+md_str+'.npy')
accs = analysis.acc_components(
    model, layer_str, vec, x_train, y_train, x_test, y_test)
if args.abs:
    np.save(model_str+'/accabs_'+layer_str+md_str, accs)
else:
    np.save(model_str+'/acc_'+layer_str+md_str, accs)


# compute loss landscape in directions of the weights and drift mode

In [ ]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import analysis
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)

parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args = parser.parse_args()
model_str = args.name
layer_str = args.layer

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)


layer_pointer = analysis.get_layer_pointer(model, layer_str)
weights = analysis.get_weights(layer_pointer)

shift_step = 0.01
epsilons = np.arange(-1, 3, shift_step)
landscape_array = analysis.loss_landscape(
    model, layer_str, weights, epsilons, x_train, y_train, x_test, y_test)
np.save(model_str+'/loss_scaling_'+layer_str, landscape_array)

pc = np.load(model_str+'/pc_'+layer_str+'.npy')
shift_step = 0.001
epsilons = np.arange(-0.05, 0.05, shift_step)
landscape_array = analysis.loss_landscape(
    model, layer_str, pc[0], epsilons, x_train, y_train, x_test, y_test)
np.save(model_str+'/loss_pc_'+layer_str, landscape_array)


# compute loss landscape in direction of Hessian eigenvectors

In [ ]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import argparse
parser = argparse.ArgumentParser(
    prog='train MLP256',
    description='train MLP256',
    epilog='Based on the rmt package.')
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args = parser.parse_args()
model_str = args.name
layer_str = args.layer

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)


evh = np.load(model_str+'/evh_'+layer_str+'.npy')


layer_pointer = analysis.get_layer_pointer(model, layer_str)
weights = analysis.get_weights(layer_pointer)

weights_prod = np.tensordot(weights, evh, axes=(0, 1))
max_entry = np.argmax(weights_prod)
step_range = 0  # evh.shape[0]

step_array = np.concatenate(([0], [max_entry]))
shift_step = 0.01
for i in step_array:
    shift = weights_prod[i]
    epsilons = np.arange(shift-weights_prod[max_entry]-shift_step,
                         2*shift_step+shift+weights_prod[max_entry], shift_step)
    landscape_array = analysis.loss_landscape(
    model, layer_str, evh[i], epsilons-shift, x_train, y_train, x_test, y_test)
    np.save(model_str+'/evh_eps_'+layer_str+'_'+str(i),
            [epsilons,landscape_array[1:]])


In [ ]:
# compute component comparison between 2 bases
import numpy as np
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, all or layer of sv",
    dest="layer"
)
parser.add_argument(
    '-m1', '--mode1',
    type=str,
    default="evh",
    help="component mode 1",
    dest="mode1"
)
parser.add_argument(
    '-m2', '--mode2',
    type=str,
    default="vc",
    help="component mode 2",
    dest="mode2"
)



args=parser.parse_args()
model_str=args.name
md_1_str=args.mode1
md_2_str=args.mode2
layer_str=args.layer

comp_1=np.load(model_str+'/'+md_1_str+'_'+layer_str+'.npy')
comp_2=np.load(model_str+'/'+md_2_str+'_'+layer_str+'.npy')
field=np.tensordot(comp_1,comp_2,axes=(1,1))
np.save(model_str+'/comp_'+md_1_str+'_'+md_2_str+'_'+str(layer_index),field)

# compute component comparison layer and full network

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, all or layer of sv",
    dest="layer"
)
parser.add_argument(
    '-m', '--mode',
    type=str,
    default="evh",
    help="type of input vectors, evh or pc",
    dest="mode"
)

parser.add_argument(
    '-lx', '--lyrdex',
    type=int,
    default="-1",
    help="layer index for singular values",
    dest="lx"
)

args=parser.parse_args()
model_str=args.name
md_str=args.mode
layer_index=args.lx
layer_str=args.layer

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)

if layer_str[:6] == "layers":
        layer_name = [getattr(model, "layers")[int(
            layer_str[7:(len(layer_str)-1)])]]
        layer_index= layer_str[7:(len(layer_str)-1)]
else:
    layer_name = [getattr(model, layer_str)]
layer_shape=layer_name.get_weights()[0].shape
layer_size=1
for i in layer_name.get_weights()[0].shape:
    layer_size*=i
weight_list=model.trainable_weights
i_start=0
for layer in weight_list[:layer_index]:
    i_start+=np.prod(layer.shape)
evh=np.load(model_str+'/'+md_str+'_all'+'.npy')[:,i_start:i_start+layer_size]
evh_layer=np.load(model_str+'/'+md_str+'_'+layer_str+'.npy')
field=np.tensordot(evh_layer,evh,axes=(1,1))
np.save(model_str+'/comp_'+md_str+'_'+str(layer_index),field)

# compute Hessian eigenvectors

In [ ]:

import tensorflow as tf
import numpy as np
import hess
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, all or layer of sv",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
ewh,evh=hess.comp_eig(model,layer_str,x_train,y_train)
np.save(model_str+'/ewh_'+layer_str,ewh)
np.save(model_str+'/evh_'+layer_str,evh)

# compute weights product


In [ ]:
import tensorflow as tf
import numpy as np
import analysis
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, all or layer of sv",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
weights_prod=analysis.evh_weights_prod(model,model_str,layer_str)
np.save(model_str+'/weights_prod_'+layer_str,weights_prod)

# wigner surmise

In [ ]:
import numpy as np
import analysis
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
args=parser.parse_args()
model_str=args.name
layer_str=args.layer
ewh=np.load(model_str+'/ewh_'+layer_str+'.npy')
# width of the gaussian in the unfolding process
average = 10
spacings = analysis.wigner(ewh, average) 
np.save(model_str+'/spacings',spacings)